In [18]:
class Node(object):
    def __init__(self, name):
        self.name = name
    def get_name(self):
        return self.name
    def __str__(self):
        return self.name

In [19]:
class Edge(object):
    def __init__(self, src, dest):
        # src and dest are node objects
        self.source = src
        self.destination = dest
    def get_source(self):
        return self.source
    def get_destination(self):
        return self.destination
    def __str__(self):
        return self.source.get_name() + '->' + self.destination.get_name()

Adjacency list is created to represent the graph. Here we put a list of destinations for every node.

In [47]:
class Digraph(object):
    def __init__(self):
        self.edges = {}
    
    def add_node(self, node): #all the nodes must be added first before adding any edges
        if node in self.edges:
            raise ValueError("Node already present")
        else:
            self.edges[node] = []
    
    def add_edge(self, edge):
        source = edge.get_source()
        destination = edge.get_destination()
        if source in self.edges and destination in self.edges: #self.edges is the graph dictionary
            self.edges[source].append(destination) #self.edges[source] is a list and destination gets appended to it
        else:
            raise ValueError("Node is not in graph")
            
    def children_of_node(self, node):
        return self.edges[node]
    
    def node_present(self, node):
        return node in self.edges
    
    def get_node(self, name):
        for node in self.edges:
            if node.get_name() == name:
                return node
        raise NameError(name)
    
    def __str__(self):
        result = ''
        for source in self.edges:
            for destination in self.edges[source]:
                result = result + source.get_name() + '->' + destination.get_name() + '\n'
        return result[:-1]

##### For graphs which can have edges going in both directions

In [40]:
class Graph(Digraph):
    def add_edge(self, edge): #edge is an edge object
        Digraph.add_edge(self, edge)
        rev = Edge(edge.get_destination(), edge.get_source()) #edge object with destination as source and vice versa
        Digraph.add_edge(self, rev)

### Making a Graph

In [41]:
def make_city_graph(graphtype):
    cities = ['Boston', 'Providence', 'New York', 'Chicago', 'Denver', 'Los Angeles', 'Phoenix']
    city_graph = graphtype()
    for city in cities:
        city_graph.add_node(Node(city))
        
    city_graph.add_edge(Edge(city_graph.get_node('Boston'), city_graph.get_node('Providence')))
    city_graph.add_edge(Edge(city_graph.get_node('Boston'), city_graph.get_node('New York')))
    city_graph.add_edge(Edge(city_graph.get_node('Providence'), city_graph.get_node('Boston')))
    city_graph.add_edge(Edge(city_graph.get_node('Providence'), city_graph.get_node('New York')))
    city_graph.add_edge(Edge(city_graph.get_node('New York'), city_graph.get_node('Chicago')))
    city_graph.add_edge(Edge(city_graph.get_node('Chicago'), city_graph.get_node('Denver')))
    city_graph.add_edge(Edge(city_graph.get_node('Chicago'), city_graph.get_node('Phoenix')))
    city_graph.add_edge(Edge(city_graph.get_node('Denver'), city_graph.get_node('Phoenix')))
    city_graph.add_edge(Edge(city_graph.get_node('Denver'), city_graph.get_node('New York')))
    city_graph.add_edge(Edge(city_graph.get_node('Los Angeles'), city_graph.get_node('Boston')))
    return city_graph

In [50]:
city_graph = make_city_graph(Digraph)

In [67]:
print(city_graph)

Boston->Providence
Boston->New York
Providence->Boston
Providence->New York
New York->Chicago
Chicago->Denver
Chicago->Phoenix
Denver->Phoenix
Denver->New York
Los Angeles->Boston


## Shortest Path in a Graph

### Depth First Search

In [154]:
def depth_first_search(graph, start, end, previous_path=[], shortest_path= None):
    previous_path = previous_path + [start] 
    if start == end:
        return previous_path #if the loop has reached the end then this path must be the best
    for city in graph.children_of_node(start): #start should be a node object
        if city not in previous_path: #to avoid loops 
            if shortest_path == None or len(previous_path) < len(shortest_path):
                new_path = depth_first_search(graph, city, end, previous_path, shortest_path)
                if new_path != None: #Cannot initialise shortest path with [] because it has to be none
                    shortest_path = new_path
    return shortest_path

In [135]:
def print_path(path):
    for node in path:
        print(node.get_name() + '->')

Shortest path is an empty list in the beginning so when the function reaches a node which goes out to nowhere, the for City in graph loop is skipped and shortest_path is returned. The first time it happens will be [] but subsequently it will become a known list.


The if statement checks if the len of previous path is greater than the shortest path so the function does not return any path if the len of the previous path is greater than the shortest path because we know that such a path cannot be the answer.

In [136]:
path = depth_first_search(city_graph, city_graph.get_node("Boston"), city_graph.get_node("Phoenix"), previous_path=[], shortest_path=None)

In [137]:
print_path(path)

Boston->
New York->
Chicago->
Phoenix->


### Breadth First Search

In [145]:
def breadth_first_search(graph, start, end):
    init_path = [start] #add node to a list
    path_queue = [init_path] #add path to the list
    
    
    while len(path_queue) != 0:
        temp_path = path_queue.pop(0) #first path out of all the paths in the queue
        #we only check the last element below and as we go ahead from the last node, we only care about it
        last_node = temp_path[-1]
        if last_node == end:
            return temp_path
        for next_node in graph.children_of_node(last_node): #we search the children of the next node
            if next_node not in temp_path:
                #the previous nodes are stored in temp_path and now we add the next node. we only care for the last node in the
                #path so its fine
                new_path = temp_path + [next_node] 
                path_queue.append(new_path) #list we're looping on is being added elements to
                
    return none


If in a list we put a while loop which runs till its length isn't zero and then in the loop we reduce the list's length by one, we sort of have a loop of a new kind. Here we can add elements to the list we are iterating on.

In [146]:
bfs_path = breadth_first_search(city_graph, city_graph.get_node('Boston'), city_graph.get_node('Phoenix'))

In [147]:
print_path(bfs_path)

Boston->
New York->
Chicago->
Phoenix->


### Weighted Graph Problem

The edge class can be modified to add weights for the edges

In [178]:
class Edge(object):
    def __init__(self, src, dest, weight):
        # src and dest are node objects
        self.source = src
        self.destination = dest
        self.weight = weight
    def get_source(self):
        return self.source
    def get_destination(self):
        return self.destination
    def get_weight(self):
        return self.weight
    def __str__(self):
        return self.source.get_name() + '->' + self.destination.get_name()

Digraph class is also modified below

In [192]:
class Digraph(object):
    def __init__(self):
        self.edges = {}
    
    def add_node(self, node): #all the nodes must be added first before adding any edges
        if node in self.edges:
            raise ValueError("Node already present")
        else:
            self.edges[node] = []
    
    def add_edge(self, edge):
        source = edge.get_source()
        destination = edge.get_destination()
        weight = edge.get_weight()
        if source in self.edges and destination in self.edges: #self.edges is the graph dictionary
            self.edges[source].append([destination, weight]) #the destination and weight are made into a list and added
        else:
            raise ValueError("Node is not in graph")
            
    def children_of_node(self, node):
        return self.edges[node]
    
    def node_present(self, node):
        return node in self.edges
    
    def get_node(self, name):
        for node in self.edges:
            if node.get_name() == name:
                return node
        raise NameError(name)
    
    def __str__(self):
        result = ''
        for source in self.edges:
            for [destination, weight] in self.edges[source]:
                result = result + source.get_name() + '->' + destination.get_name() + '\n'
        return result[:-1]

The graph can be made now with the weights

In [256]:
def make_city_graph(graphtype):
    cities = ['Boston', 'Providence', 'New York', 'Chicago', 'Denver', 'Los Angeles', 'Phoenix']
    city_graph = graphtype()
    for city in cities:
        city_graph.add_node(Node(city))
        
    city_graph.add_edge(Edge(city_graph.get_node('Boston'), city_graph.get_node('Providence'), 0))
    city_graph.add_edge(Edge(city_graph.get_node('Boston'), city_graph.get_node('New York'), 10))
    city_graph.add_edge(Edge(city_graph.get_node('Providence'), city_graph.get_node('Boston'), 10))
    city_graph.add_edge(Edge(city_graph.get_node('Providence'), city_graph.get_node('New York'), 2))
    city_graph.add_edge(Edge(city_graph.get_node('New York'), city_graph.get_node('Chicago'), 1))
    city_graph.add_edge(Edge(city_graph.get_node('Chicago'), city_graph.get_node('Denver'), 1))
    city_graph.add_edge(Edge(city_graph.get_node('Chicago'), city_graph.get_node('Phoenix'), 10))
    city_graph.add_edge(Edge(city_graph.get_node('Denver'), city_graph.get_node('Phoenix'), 1))
    city_graph.add_edge(Edge(city_graph.get_node('Denver'), city_graph.get_node('New York'), 10))
    city_graph.add_edge(Edge(city_graph.get_node('Los Angeles'), city_graph.get_node('Boston'), 10))
    return city_graph

In [257]:
weighted_city_graph = make_city_graph(Digraph)

In [258]:
print(weighted_city_graph)

Boston->Providence
Boston->New York
Providence->Boston
Providence->New York
New York->Chicago
Chicago->Denver
Chicago->Phoenix
Denver->Phoenix
Denver->New York
Los Angeles->Boston


In [222]:
{'Boston':[['Providence', 10], ['New York',1]], 'Providence': [['Boston', 10],['New York',10]], 'New York':[['Chicago', 1]], 'Chicago': [['Denver', 1], ['Phoenix', 10]], 'Denver': [['Phoenix', 1],['New York',10]], 'Los Angeles':[['Boston',10]]}

{'Boston': [['Providence', 10], ['New York', 1]],
 'Providence': [['Boston', 10], ['New York', 10]],
 'New York': [['Chicago', 1]],
 'Chicago': [['Denver', 1], ['Phoenix', 10]],
 'Denver': [['Phoenix', 1], ['New York', 10]],
 'Los Angeles': [['Boston', 10]]}

Weighted Depth First Search

In [259]:
def depth_first_search(graph, start, end, previous_path=[], shortest_path= None):
    previous_path = previous_path + [start] 
    if start == end:
        return previous_path #if the loop has reached the end then this path must be the best
    for city in graph.children_of_node(start): #start should be a node object
        if city not in previous_path: #to avoid loops 
            if shortest_path == None or len(previous_path) < len(shortest_path):
                new_path = depth_first_search(graph, city, end, previous_path, shortest_path)
                if new_path != None: #Cannot initialise shortest path with [] because it has to be none
                    shortest_path = new_path
    return shortest_path

In [260]:
def weighted_dfs(graph, start, end, previous_path= [], weights = [], best_path = None, best_path_weights=[], toPrint = False):
    previous_path = previous_path + [start]
    if start == end:
        return previous_path, weights
    for [node, weight] in graph.children_of_node(start):
        if node not in previous_path:
            weights = weights + [weight]
            if best_path == None or sum(weights) < sum(best_path_weights):
                new_path, new_weights = weighted_dfs(graph, node, end, previous_path, weights, best_path, best_path_weights)
                if new_path != None:
                    best_path = new_path
                    best_path_weights = new_weights
    return best_path, best_path_weights

When we reach the end of a branch, the for loop gets skipped and the best_path and best_path_weights get returned and become new_path and new_weights respectively. As new_value is not equal to None, these values get stored as best_path and best_path_weights. Next time when we go to the for loop to explore another path to the destination, the condition sum(weights)<sum(best_path_weights) makes sure that we only follow paths which have weights less than our path.

In [261]:
boston_node = weighted_city_graph.get_node('Boston')
phoenix_node = weighted_city_graph.get_node('Phoenix')

In [262]:
weighted_path, weights = weighted_dfs(weighted_city_graph, boston_node, phoenix_node, previous_path=[], weights=[], best_path=None, best_path_weights=[])

In [263]:
print_path(weighted_path)

Boston->
Providence->
New York->
Chicago->
Denver->
Phoenix->
